In [162]:

import os
import re
import glob
import datetime
import warnings
import pandas as pd
import numpy as np
from fuzzywuzzy import process

import sys
import warnings
sys.path.insert(0, '../') 
from utils import init_logger

def common_subheaders()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        ('Advertising, Public Relations and Marketing ',
        'Air Transportation',
        'Amusement and Recreation',
        'Apparel Manufacturing',
        'Building Equipment Contractors',
        'Business Support Services',
        'Chemicals',
        'Communications Equipment Manufacturing',
        'Credit Related Activities',
        'Computer Systems Design and Related Services',
        'Credit (Nondepository)',
        'Data Processing and Hosting Services',
        'Educational Support Services',
        'Electronic Component Manufacturing',
        'Equipment Leasing',
        'Facilities Support Services',
        'Grocery Stores',
        'Hospitals',
        'Insurance',
        'Lessors of Nonfinancial Licenses',
        'Management, Scientific, and Technical Consulting Services',
        'Motion Picture and Video Industries',
        'Other Information Services',
        'Other Manufacturing',
        'Other Publishing',
        'Other Real Estate Activities',
        'Other Telecommunications',
        'Plastics Manufacturing',
        'Radio and Television Broadcasting',
        'Real Estate Leasing',
        'Restaurants',
        'Retail',
        'Satellite Telecommunications',
        'Scientific Research and Development Services',
        'Texttile Furnishings Mills',
        'Traveler Arrangement',
        'Software Publishing',
        'Utility System Construction',
        'Wholesalers',
        'Wired Telecommunications Carriers',
        'Wireless Telecommunications Carriers',
        )
    ))


def standard_field_names()->tuple:
    return (
        'portfolio_company',
        # 'Portfolio Company /Principal Business',
        'investment_/interest_rate_/maturity',
        'percentage__interest/__shares',
        # 'Principal',
        'cost',
        'value',
        'short-term_investments',
        'percentage_ownership',
        'percent_of_class_held',
        # 'Investment',
        'percent_of_interests_held',
        # 'Industry',
        'spread_above_index',
        'aquisition_date',
        # 'Maturity',
        # 'Principal/Shares',
        # 'Investment Type',
        'of_Net_Assets',
        # 'business description',
        # 'type of investment',
        # 'investment date',
        'reference_rate_and_spread',
        'pik_rate',
        # 'maturity date',
        # 'cost',
        'footnotes',
        # 'industry',
        # 'principal amount',
        # 'fair value',
    )


def company_control_headers()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        (
        'Debt Investments',
        'Debt Investments (82.23%)',
        'Debt Investments (A)',
        'Debt Investments (continued)',
        'Equity Securities',
        'Equity Securities (continued)',
        'Cash and Cash Equivalents',
        )
    ))


def except_rows()->tuple:
    return (
        'Debt_Securities_and_Bond_Portfolio',
        'CLO__Fund_Securities',
        'CLO__Fund_Securities',
        'Debt_Securities_Portfolio',
        'CLO_Investment'
    )

# https://www.sec.gov/robots.txt
def get_standard_name(col, choices, score_cutoff=60):
    best_match, score = process.extractOne(col, choices)
    if score > score_cutoff:
        return best_match
    return col

def stopping_criterion(qtr:str)->str:
    return '{}'.format(r'Total_*Investments')


def concat(*dfs)->list:
    final = []
    for df in dfs:
        final.extend(df.values.tolist())
    return final

    
def get_key_fields(
    df_cur:pd.DataFrame,
)->tuple:
    important_fields = standard_field_names() + common_subheaders()
    for idx,row in enumerate(df_cur.iterrows()):
        found = any(any(
            key in str(field).lower() 
            for key in important_fields)
                    for field in row[-1].dropna().tolist()
            )
        if found and len(set(row[-1].dropna().tolist())) >= 6:
            cols = df_cur.iloc[:idx + 1].apply(lambda row: ' '.join(row.dropna()), axis=0).tolist()
            fields = strip_string(cols,standardize=found) 
            return fields
    return strip_string(df_cur.iloc[0].tolist())

def strip_string(
    columns_names:list,
    standardize:bool=False,
)->tuple:
    # columns = tuple(map(lambda col:re.sub(r'[^a-z]', '', str(col).lower()),columns_names))
    if standardize:
        standard_fields = standard_field_names()
        return tuple(
            re.sub(r'[^a-zA-Z]', '_',get_standard_name(str(col).strip().lower(),standard_fields)) for col in columns_names
        )
    return tuple(re.sub(r'[^a-zA-Z]', '_',str(col).strip().lower()) for col in columns_names)


# Function to extract date and convert to datetime object
def extract_date(file_path):
    # Extract date from file path (assuming date is always in 'YYYY-MM-DD' format)
    date_str = re.search(r'\d{4}-\d{2}-\d{2}', file_path).group()
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')


def merge_duplicate_columns(
    df:pd.DataFrame,
    merged_pair_idxs:dict={}
)->pd.DataFrame:
    duplicate_cols = merged_pair_idxs.keys()
    flag = not merged_pair_idxs.keys()
    if flag: 
        duplicate_cols = df.columns.unique() 
    for col_name in duplicate_cols:
        # display(col_name)
        mask = merged_pair_idxs.get(col_name)
        if flag:
            mask = df.columns == col_name
            merged_pair_idxs[col_name] = mask
        duplicate_data = df.loc[:, mask]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, ~mask]
        df[col_name] = merged_data
        # display(df)
    return df.reset_index(drop=True),merged_pair_idxs

def extract_subheaders(
    df:pd.DataFrame,
    control:bool,
)->pd.DataFrame:
    col_name = 'company_control' if control else 'Type_of_Investment'
    if col_name in df.columns:
        return df
    include = df.apply(
        lambda row: re.search('|'.join(company_control_headers() if control else common_subheaders()), str(row[0]), re.IGNORECASE) is not None,
        axis=1
    )  
    
    exclude = ~df.apply(
        lambda row: row.astype(str).str.contains('total|Inc|Ltd|LLC|Holdings|LP|Co|Corporation', case=False, na=False).any(),
        axis=1
    )
    idx = df[include & exclude].index.tolist()
    df[col_name] = None
    if not idx:
        return df

    prev_header = subheader = None
    df.loc[idx[-1]:,col_name] = df.iloc[idx[-1],1] if isinstance(df.iloc[idx[-1],0],float)  else df.iloc[idx[-1],0]
    for j,i in enumerate(idx[:-1]):
        prev_header = subheader
        subheader = df.iloc[i,1] if isinstance(df.iloc[i,0],float)  else df.iloc[i,0]
        df.loc[idx[j]:idx[j+1],col_name] = subheader if subheader != '' else prev_header
    return df


def remove_row_duplicates(row:pd.Series)->pd.Series: 
    out = []
    for v in row:
        if v in out and not str(v).replace('$','').isnumeric():
            out.append(np.nan)
        else:
            out.append(v)
    return pd.Series(out)


def _clean(
    file_path:str,
    except_rows:str,
    merged_pair_idxs:dict={},
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0,na_values=[' ', ''])
    df.replace(to_replace=r'[\[\](){},$%˄\xa0\u200b]', value='', regex=True,inplace=True)
    df.replace(['Principal_Business',' '],'_',regex=True,inplace=True)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.dropna(axis=0,how='all',inplace=True)
    df = df[~df.apply(lambda row:row.astype(str).str.contains(except_rows,case=False, na=False).any(),axis=1)]
    if not df.apply(lambda col: col.astype(str).str.contains(r'total_investments', case=False, regex=True)).any().any() and df.shape[0] < 3:
        return pd.DataFrame(), merged_pair_idxs
    
    duplicate_idx = df.apply(lambda row:row[pd.to_numeric(row,errors='coerce').isna()].duplicated().sum() > 1 ,axis=1)
    clean_rows = df.loc[duplicate_idx].apply(remove_row_duplicates, axis=1).reset_index(drop=True)
    j = 0
    # display(clean_rows)
    for i,flag in enumerate(duplicate_idx):
        if not flag:
            continue
        df.iloc[i,:] = clean_rows.loc[j,:].tolist()
        j += 1
    if not merged_pair_idxs:
        important_fields = strip_string(get_header_rows(df),standardize=True)#get_key_fields(df)
        df.columns = important_fields
    df,merge_pair_idxs = merge_duplicate_columns(df,merged_pair_idxs=merged_pair_idxs)

    df.replace([''],np.nan,regex=True,inplace=True) #':','$','%'
    df.dropna(axis=1,how='all',inplace=True)
    # columns = (~df.isna()).sum(axis=0) < (6 if df.shape[0] > 10 else 2 if df.shape[0] <= 4 else 0)
    columns = [col.isdigit() for col in df.columns]
    df = df.drop(columns=df.columns[columns])
    return df.reset_index(drop=True),merge_pair_idxs




def get_header_rows(
    df_cur:pd.DataFrame,
)->tuple:
    for idx,row in df_cur.reset_index().iterrows():
        found = any(str(v).replace("$",'').replace("%",'').isnumeric() for v in row)
        if found:     
            out = df_cur.iloc[:idx + 1,:].apply(
                lambda row: ' '.join(
                    row[row.notna()].astype(str).values
            ), axis=0)
            return out
    
    return strip_string(df_cur.iloc[0].tolist())


def main()->None:
    qtrs = os.listdir('.')
    ex = exceptions()
    ex_rows = '|'.join(except_rows())
    for qtr in qtrs:
        if '.csv' in qtr or os.path.exists(os.path.join(qtr,'output',f"{qtr}.csv")) or not os.path.exists(os.path.join(qtr,f'Schedule_of_Investments_0.csv')):
            continue
        # qtr = '2022-06-30'
        logger.info(qtr)
        index_list_sum = i = 0
        soi_files = sorted([
            os.path.join(qtr,file) 
            for file in os.listdir(qtr)
            if file.endswith('.csv')
        ],key=lambda f: int(f.split('_')[-1].split('.')[0]))
        # soi_files = [f for f in soi_files] # if f not in ex]
        if len(soi_files) == 0:
            continue
        merged_pair_idxs = ex.get(soi_files[i],{})
        df,merged_pair_idxs = _clean(soi_files[i],except_rows=ex_rows,merged_pair_idxs=merged_pair_idxs)

        index_list = df.apply(
            lambda row:row.astype(str).str.contains(stopping_criterion(qtr), case=False, na=False).any(),
            axis=1
        )
        index_list_sum = index_list.sum()
        dfs = [df]     
        i += 1

        while index_list_sum == 0:
            logger.info(soi_files[i])
            merged_pair_idxs = ex.get(soi_files[i],{})
            # display(soi_files[i])

            # display(merged_pair_idxs)
            df,merged_pair_idxs = _clean(soi_files[i],except_rows=ex_rows,merged_pair_idxs=merged_pair_idxs)
            dfs.append(df)
            index_list = df.apply(
                lambda row:row.astype(str).str.contains(stopping_criterion(qtr), case=False, na=False).any(),
                axis=1
            )
            index_list_sum = index_list.sum()
            i += 1
        date_final = dfs[0]
        if len(dfs) > 1:
            date_final = pd.concat(dfs,axis=0,ignore_index=True)#pd.DataFrame(concat(*dfs))
        # date_final = extract_subheaders(date_final,control=True)
        # date_final = extract_subheaders(date_final,control=False)

        date_final['qtr'] = qtr.split('/')[-1]
        if not os.path.exists(os.path.join(qtr,'output')):
            os.makedirs(os.path.join(qtr,'output'))
        columns_to_drop = date_final.notna().sum() <= 2
        date_final.drop(columns=columns_to_drop[columns_to_drop].index)
        # print(date_final.columns)
        date_final.to_csv(os.path.join(qtr,'output',f'{qtr}.csv'),index=False)
        # break
    
    # Use glob to find files
    files = sorted(glob.glob(f'*/output/*.csv'), key=extract_date)
    single_truth = pd.concat([
        pd.read_csv(df) for df in files
    ],axis=0,ignore_index=True)
    single_truth.drop(columns=single_truth.columns[['Unnamed' in col for col in single_truth.columns]],inplace=True)
    # important_fields = strip_string(get_header_rows(single_truth),standardize=True)#get_key_fields(df)
    # single_truth.columns = important_fields
    # single_truth,_ = merge_duplicate_columns(single_truth,merged_pair_idxs={})
    single_truth.to_csv(f'{cik}_soi_table.csv',index=False)
    

def exceptions()->dict:
    return {
        '2021-12-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2021-12-31/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2021-12-31/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2021-12-31/Schedule_of_Investments_4.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2021-12-31/Schedule_of_Investments_5.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2021-12-31/Schedule_of_Investments_6.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2021-12-31/Schedule_of_Investments_7.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2021-12-31/Schedule_of_Investments_8.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False]+[True,True]+[False]*26),
            'interest_rate':np.array([False,False,False]+[True,True,True]+[False]*23),
            'initial_acquisition':np.array([False]*4+[True]+[False]*22),
            'maturity':np.array([False]*5+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*7+[True]+[False]*19),  
            'cost':np.array([False]*10+[True]+[False]*16),
            'fair_value':np.array([False]*13+[True]+[False]*13),
            'percentage_of_class':np.array([False]*15+[True]+[False]*11),
        },
        '2018-12-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*24),
            'industry':np.array([False,True]+[False]*23),
            'security___':np.array([False,False]+[True]+[False]*22),
            'footnotes':np.array([False,False,False]+[True]+[False]*21),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*20),
            'aquisition_date':np.array([False]*5+[True]+[False]*19),
            'maturity':np.array([False]*6+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*16),  
            'cost':np.array([False]*11+[True]+[False]*13),
            'value':np.array([False]*14+[True]+[False]*10),
            '':np.array([True]*15+[False]*10)
        },
        '2019-03-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True,True]+[False]*25),
            'industry':np.array([True]+[False]*25),
            'security___':np.array([False,True]+[False]*24),
            'footnotes':np.array([False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False]+[True,True]+[False]*21),
            'aquisition_date':np.array([False]*5+[True]+[False]*19),
            'maturity':np.array([False]*6+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*16),  
            'cost':np.array([False]*11+[True]+[False]*13),
            'value':np.array([False]*14+[True]+[False]*10),
            '':np.array([True]*15 + [False]*10),
        },
        '2019-06-30/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True,True]+[False]*25),
            'industry':np.array([True]+[False]*25),
            'security___':np.array([False,True]+[False]*24),
            'footnotes':np.array([False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False]+[True,True]+[False]*21),
            'aquisition_date':np.array([False]*5+[True]+[False]*19),
            'maturity':np.array([False]*6+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*16),  
            'cost':np.array([False]*11+[True]+[False]*13),
            'value':np.array([False]*14+[True]+[False]*10),
            '':np.array([True]*15 + [False]*10),
        },
        '2019-09-30/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True,True]+[False]*25),
            'industry':np.array([True]+[False]*25),
            'security___':np.array([False,True]+[False]*24),
            'footnotes':np.array([False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False]+[True,True]+[False]*21),
            'aquisition_date':np.array([False]*5+[True]+[False]*19),
            'maturity':np.array([False]*6+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*16),  
            'cost':np.array([False]*11+[True]+[False]*13),
            'value':np.array([False]*14+[True]+[False]*10),
            '':np.array([True]*15 + [False]*10),
        },
        '2019-12-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*23),
            'industry':np.array([True]+[False]*23),
            'security___':np.array([False,True]+[False]*22),
            'footnotes':np.array([False,False]+[True]+[False]*21),
            'investment__interest_rate__maturity':np.array([False,False,False]+[True]+[False]*20),
            'aquisition_date':np.array([False]*4+[True]+[False]*19),
            'maturity':np.array([False]*5+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*7+[True]+[False]*16),  
            'cost':np.array([False]*10+[True]+[False]*13),
            'value':np.array([False]*13+[True]+[False]*10),
            '':np.array([True]*6 + [False]*18),
        },
        '2017-12-31/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'fair_value':np.array([False]*20+[True]+[False]*6),
            'of_nav':np.array([False]*23+[True]+[False]*3),
            'cost':np.array([False]*16+[True]+[False]*10),
        },
        '2020-03-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True,True]+[False]*21),
            'aquisition_date':np.array([False]*6+[True]+[False]*19),
            'maturity':np.array([False]*7+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*9+[True]+[False]*16),  
            'cost':np.array([False]*12+[True]+[False]*13),
            'value':np.array([False]*15+[True]+[False]*10),
            '':np.array([True]*14 + [False]*12),
        },
        '2020-09-30/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True,True]+[False]*21),
            'aquisition_date':np.array([False]*6+[True]+[False]*19),
            'maturity':np.array([False]*7+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*9+[True]+[False]*16),  
            'cost':np.array([False]*12+[True]+[False]*13),
            'value':np.array([False]*15+[True]+[False]*10),
            '':np.array([True]*14 + [False]*12),
        },
        '2020-06-30/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True,True]+[False]*21),
            'aquisition_date':np.array([False]*6+[True]+[False]*19),
            'maturity':np.array([False]*7+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*9+[True]+[False]*16),  
            'cost':np.array([False]*12+[True]+[False]*13),
            'value':np.array([False]*15+[True]+[False]*10),
            '':np.array([True]*14 + [False]*12),
        },
        '2020-12-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True,True]+[False]*21),
            'aquisition_date':np.array([False]*6+[True]+[False]*19),
            'maturity':np.array([False]*7+[True]+[False]*18), 
            'par_amount___quantity':np.array([False]*9+[True]+[False]*16),  
            'cost':np.array([False]*12+[True]+[False]*13),
            'value':np.array([False]*15+[True]+[False]*10),
            '':np.array([True]*14 + [False]*12),
        },
        '2021-03-31/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security':np.array([False,False]+[True]+[False]*24),
            'notes':np.array([False,False,False]+[True]+[False]*23),
            'interest_rate':np.array([False,False,False,False]+[True]+[False]*22),
            'initial_acquisition':np.array([False]*7+[True]+[False]*19),
            'maturity':np.array([False]*8+[True]+[False]*18), 
            'par_amount_quantity':np.array([False]*10+[True]+[False]*16),  
            'cost':np.array([False]*13+[True]+[False]*13),
            'fair_value':np.array([False]*16+[True]+[False]*10),
        },
        '2021-03-31/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security':np.array([False,False]+[True]+[False]*24),
            'notes':np.array([False,False,False]+[True]+[False]*23),
            'interest_rate':np.array([False,False,False,False]+[True]+[False]*22),
            'initial_acquisition':np.array([False]*7+[True]+[False]*19),
            'maturity':np.array([False]*8+[True]+[False]*18), 
            'par_amount_quantity':np.array([False]*10+[True]+[False]*16),  
            'cost':np.array([False]*13+[True]+[False]*13),
            'fair_value':np.array([False]*16+[True]+[False]*10),
        },
        '2021-06-30/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security':np.array([False,False]+[True]+[False]*24),
            'notes':np.array([False,False,False]+[True]+[False]*23),
            'interest_rate':np.array([False,False,False,False]+[True]+[False]*22),
            'initial_acquisition':np.array([False]*7+[True]+[False]*19),
            'maturity':np.array([False]*8+[True]+[False]*18), 
            'par_amount_quantity':np.array([False]*10+[True]+[False]*16),  
            'cost':np.array([False]*13+[True]+[False]*13),
            'fair_value':np.array([False]*16+[True]+[False]*10),
        },
        '2021-06-30/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security':np.array([False,False]+[True]+[False]*24),
            'notes':np.array([False,False,False]+[True]+[False]*23),
            'interest_rate':np.array([False,False,False,False]+[True]+[False]*22),
            'initial_acquisition':np.array([False]*7+[True]+[False]*19),
            'maturity':np.array([False]*8+[True]+[False]*18), 
            'par_amount_quantity':np.array([False]*10+[True]+[False]*16),  
            'cost':np.array([False]*13+[True]+[False]*13),
            'fair_value':np.array([False]*16+[True]+[False]*10),
        },
        '2021-06-30/Schedule_of_Investments_4.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security':np.array([False,False]+[True]+[False]*24),
            'notes':np.array([False,False,False]+[True]+[False]*23),
            'interest_rate':np.array([False,False,False,False]+[True]+[False]*22),
            'initial_acquisition':np.array([False]*7+[True]+[False]*19),
            'maturity':np.array([False]*8+[True]+[False]*18), 
            'par_amount_quantity':np.array([False]*10+[True]+[False]*16),  
            'cost':np.array([False]*13+[True]+[False]*13),
            'fair_value':np.array([False]*16+[True]+[False]*10),
        },
        '2020-12-31/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security':np.array([False,False]+[True]+[False]*24),
            'notes':np.array([False,False,False]+[True]+[False]*23),
            'interest_rate':np.array([False,False,False,False]+[True]+[False]*22),
            'initial_acquisition':np.array([False]*7+[True]+[False]*19),
            'maturity':np.array([False]*8+[True]+[False]*18), 
            'par_amount_quantity':np.array([False]*10+[True]+[False]*16),  
            'cost':np.array([False]*13+[True]+[False]*13),
            'fair_value':np.array([False]*16+[True]+[False]*10),
        },
        '2021-09-30/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False,False]+[True]+[False]*26),
            'interest_rate':np.array([False,False,False,False]+[True,True]+[False]*24),
            'initial_acquisition':np.array([False]*6+[True]+[False]*22),
            'maturity':np.array([False]*7+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*9+[True]+[False]*19),  
            'cost':np.array([False]*12+[True]+[False]*16),
            'fair_value':np.array([False]*15+[True]+[False]*13),
        },
        '2021-09-30/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False,False]+[True]+[False]*26),
            'interest_rate':np.array([False,False,False,False]+[True,True]+[False]*24),
            'initial_acquisition':np.array([False]*6+[True]+[False]*22),
            'maturity':np.array([False]*7+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*9+[True]+[False]*19),  
            'cost':np.array([False]*12+[True]+[False]*16),
            'fair_value':np.array([False]*15+[True]+[False]*13),
        },
        '2021-09-30/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False,False]+[True]+[False]*26),
            'interest_rate':np.array([False,False,False,False]+[True,True]+[False]*24),
            'initial_acquisition':np.array([False]*6+[True]+[False]*22),
            'maturity':np.array([False]*7+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*9+[True]+[False]*19),  
            'cost':np.array([False]*12+[True]+[False]*16),
            'fair_value':np.array([False]*15+[True]+[False]*13),
        },
        '2021-09-30/Schedule_of_Investments_4.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False,False]+[True]+[False]*26),
            'interest_rate':np.array([False,False,False,False]+[True,True]+[False]*24),
            'initial_acquisition':np.array([False]*6+[True]+[False]*22),
            'maturity':np.array([False]*7+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*9+[True]+[False]*19),  
            'cost':np.array([False]*12+[True]+[False]*16),
            'fair_value':np.array([False]*15+[True]+[False]*13),
        },
        '2021-09-30/Schedule_of_Investments_5.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False,False]+[True]+[False]*26),
            'interest_rate':np.array([False,False,False,False]+[True,True]+[False]*24),
            'initial_acquisition':np.array([False]*6+[True]+[False]*22),
            'maturity':np.array([False]*7+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*9+[True]+[False]*19),  
            'cost':np.array([False]*12+[True]+[False]*16),
            'fair_value':np.array([False]*15+[True]+[False]*13),
        },
        '2021-09-30/Schedule_of_Investments_6.csv':{
            'portfolio_company':np.array([True]+[False]*29),
            'industry':np.array([False,True]+[False]*28),
            'security':np.array([False,False]+[True]+[False]*27),
            'notes':np.array([False,False,False]+[True]+[False]*26),
            'interest_rate':np.array([False,False,False,False]+[True,True]+[False]*24),
            'initial_acquisition':np.array([False]*6+[True]+[False]*22),
            'maturity':np.array([False]*7+[True]+[False]*21), 
            'par_amount_quantity':np.array([False]*9+[True]+[False]*19),  
            'cost':np.array([False]*12+[True]+[False]*16),
            'fair_value':np.array([False]*15+[True]+[False]*13),
        },
        '2022-06-30/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security':np.array([False,False]+[True]+[False]*26),
            'notes':np.array([False,False,False]+[True]+[False]*25),
            'interest_rate':np.array([False,False,False,False]+[True]+[False]*24),
            'initial_acquisition':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount_quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'fair_value':np.array([False]*14+[True]+[False]*14),
        },
        '2016-12-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*24),
            'industry':np.array([False,True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*18),
            'par_amount_quantity':np.array([False]*8+[True]+[False]*16), 
            'cost':np.array([False]*11+[True]+[False]*13),
            'fair_value':np.array([False]*14+[True]+[False]*10),
            'of_nav':np.array([False]*17+[True]+[False]*7),
        },
        '2023-09-30/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*17+[True]+[False]*11),
            '':np.array([True]*18 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*17+[True]+[False]*11),
            '':np.array([True]*18 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*17+[True]+[False]*11),
            '':np.array([True]*18 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*17+[True]+[False]*11),
            '':np.array([True]*18 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_4.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*22),
            'aquisition_date':np.array([False]*5+[True]+[False]*21),
            'maturity':np.array([False]*6+[True]+[False]*20), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*18),  
            'cost':np.array([False]*11+[True]+[False]*15),
            'value':np.array([False]*14+[True]+[False]*12),
            'percentage_of_class':np.array([False]*16+[True]+[False]*10),
            '':np.array([True]*16 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_5.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*22),
            'aquisition_date':np.array([False]*5+[True]+[False]*21),
            'maturity':np.array([False]*6+[True]+[False]*20), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*18),  
            'cost':np.array([False]*11+[True]+[False]*15),
            'value':np.array([False]*14+[True]+[False]*12),
            'percentage_of_class':np.array([False]*16+[True]+[False]*10),
            '':np.array([True]*16 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_6.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*22),
            'aquisition_date':np.array([False]*5+[True]+[False]*21),
            'maturity':np.array([False]*6+[True]+[False]*20), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*18),  
            'cost':np.array([False]*11+[True]+[False]*15),
            'value':np.array([False]*14+[True]+[False]*12),
            'percentage_of_class':np.array([False]*16+[True]+[False]*10),
            '':np.array([True]*16 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_7.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*22),
            'aquisition_date':np.array([False]*5+[True]+[False]*21),
            'maturity':np.array([False]*6+[True]+[False]*20), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*18),  
            'cost':np.array([False]*11+[True]+[False]*15),
            'value':np.array([False]*14+[True]+[False]*12),
            'percentage_of_class':np.array([False]*16+[True]+[False]*10),
            '':np.array([True]*16 + [False]*11),
        },
        '2023-09-30/Schedule_of_Investments_8.csv':{
            'portfolio_company':np.array([True]+[False]*8),
            'value':np.array([False]*1+[True,True]+[False]*6),
            'percentage_of_class':np.array([False]*3+[True,True]+[False]*3),
        },
        '2021-03-31/Schedule_of_Investments_0.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*22),
            'aquisition_date':np.array([False]*5+[True]+[False]*21),
            'maturity':np.array([False]*6+[True]+[False]*20), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*18),  
            'cost':np.array([False]*11+[True]+[False]*15),
            'value':np.array([False]*14+[True]+[False]*12),
            'percentage_of_class':np.array([False]*16+[True]+[False]*10),
        },
        '2021-03-31/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*22),
            'aquisition_date':np.array([False]*5+[True]+[False]*21),
            'maturity':np.array([False]*6+[True]+[False]*20), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*18),  
            'cost':np.array([False]*11+[True]+[False]*15),
            'value':np.array([False]*14+[True]+[False]*12),
            'percentage_of_class':np.array([False]*16+[True]+[False]*10),
        },
        '2021-03-31/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*26),
            'industry':np.array([False,True]+[False]*25),
            'security___':np.array([False,False,True]+[False]*24),
            'footnotes':np.array([False,False,False]+[True]+[False]*23),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*22),
            'aquisition_date':np.array([False]*5+[True]+[False]*21),
            'maturity':np.array([False]*6+[True]+[False]*20), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*18),  
            'cost':np.array([False]*11+[True]+[False]*15),
            'value':np.array([False]*14+[True]+[False]*12),
            'percentage_of_class':np.array([False]*16+[True]+[False]*10),
        },
        '2022-06-30/Schedule_of_Investments_0.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*16+[True]+[False]*12),
        },
        '2022-06-30/Schedule_of_Investments_1.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*16+[True]+[False]*12),
        },
        '2022-06-30/Schedule_of_Investments_2.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*16+[True]+[False]*12),
        },
        '2022-06-30/Schedule_of_Investments_3.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*16+[True]+[False]*12),
        },
        '2022-06-30/Schedule_of_Investments_4.csv':{
            'portfolio_company':np.array([True]+[False]*28),
            'industry':np.array([False,True]+[False]*27),
            'security___':np.array([False,False,True]+[False]*26),
            'footnotes':np.array([False,False,False]+[True]+[False]*25),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*24),
            'aquisition_date':np.array([False]*5+[True]+[False]*23),
            'maturity':np.array([False]*6+[True]+[False]*22), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*20),  
            'cost':np.array([False]*11+[True]+[False]*17),
            'value':np.array([False]*14+[True]+[False]*14),
            'percentage_of_class':np.array([False]*16+[True]+[False]*12),
        },
        '2022-06-30/Schedule_of_Investments_5.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
        '2022-06-30/Schedule_of_Investments_6.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
        '2022-06-30/Schedule_of_Investments_7.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
        '2022-06-30/Schedule_of_Investments_8.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
        '2022-06-30/Schedule_of_Investments_9.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
        '2022-06-30/Schedule_of_Investments_10.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
        '2022-06-30/Schedule_of_Investments_11.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
        '2022-06-30/Schedule_of_Investments_12.csv':{
            'portfolio_company':np.array([True]+[False]*27),
            'industry':np.array([False,True]+[False]*26),
            'security___':np.array([False,False,True]+[False]*25),
            'footnotes':np.array([False,False,False]+[True]+[False]*24),
            'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
            'aquisition_date':np.array([False]*5+[True]+[False]*22),
            'maturity':np.array([False]*6+[True]+[False]*21), 
            'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
            'cost':np.array([False]*11+[True]+[False]*16),
            'value':np.array([False]*14+[True]+[False]*13),
            'percentage_of_class':np.array([False]*16+[True]+[False]*11),
        },
    }


warnings.simplefilter(action='ignore', category=FutureWarning)
cik = 1675033
logger = init_logger(cik)
logger.info(cik)
main()


[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[CIK==1675033:INFO] Initializing ok.
[

INFO:CIK==1675033:Initializing ok.


[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033
[CIK==1675033:INFO] 1675033


INFO:CIK==1675033:1675033


[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31
[CIK==1675033:INFO] 2021-12-31


...

INFO:CIK==1675033:2024-06-30/Schedule_of_Investments_2.csv


[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_3.csv
[CIK==1675033:INFO] 2024

INFO:CIK==1675033:2024-06-30/Schedule_of_Investments_3.csv


[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024-06-30/Schedule_of_Investments_4.csv
[CIK==1675033:INFO] 2024

INFO:CIK==1675033:2024-06-30/Schedule_of_Investments_4.csv


In [157]:
file_path = r'2022-06-30/Schedule_of_Investments_5.csv'

def _clean(
    file_path:str,
    except_rows:str,
    merged_pair_idxs:dict={},
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0,na_values=[' ', ''])
    df.replace(to_replace=r'[\[\](){},$%˄\xa0\u200b]', value='', regex=True,inplace=True)
    df.replace(['Principal_Business',' '],'_',regex=True,inplace=True)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.dropna(axis=0,how='all',inplace=True)
    df = df[~df.apply(lambda row:row.astype(str).str.contains(except_rows,case=False, na=False).any(),axis=1)]
    if not df.apply(lambda col: col.astype(str).str.contains(r'total_investments', case=False, regex=True)).any().any() and df.shape[0] < 3:
        return pd.DataFrame(), merged_pair_idxs
    
    duplicate_idx = df.apply(lambda row:row[pd.to_numeric(row,errors='coerce').isna()].duplicated().sum() > 1 ,axis=1)
    clean_rows = df.loc[duplicate_idx].apply(remove_row_duplicates, axis=1).reset_index(drop=True)
    j = 0
    # display(clean_rows)
    for i,flag in enumerate(duplicate_idx):
        if not flag:
            continue
        df.iloc[i,:] = clean_rows.loc[j,:].tolist()
        j += 1
    if not merged_pair_idxs:
        important_fields = strip_string(get_header_rows(df),standardize=True)#get_key_fields(df)
        df.columns = important_fields
    df,merge_pair_idxs = merge_duplicate_columns(df,merged_pair_idxs=merged_pair_idxs)

    df.replace([''],np.nan,regex=True,inplace=True) #':','$','%'
    df.dropna(axis=1,how='all',inplace=True)
    # columns = (~df.isna()).sum(axis=0) < (6 if df.shape[0] > 10 else 2 if df.shape[0] <= 4 else 0)
    columns = [col.isdigit() for col in df.columns]
    df = df.drop(columns=df.columns[columns])
    return df.reset_index(drop=True),merge_pair_idxs




def merge_duplicate_columns(
    df:pd.DataFrame,
    merged_pair_idxs:dict={}
)->pd.DataFrame:
    duplicate_cols = merged_pair_idxs.keys()
    flag = not merged_pair_idxs.keys()
    if flag: 
        duplicate_cols = df.columns.unique() 
    for col_name in duplicate_cols:
        display(col_name)
        mask = merged_pair_idxs.get(col_name)
        if flag:
            mask = df.columns == col_name
            merged_pair_idxs[col_name] = mask
        duplicate_data = df.loc[:, mask]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, ~mask]
        df[col_name] = merged_data
        display(df.iloc[:,:])
    return df.reset_index(drop=True),merged_pair_idxs

merged_pair_idxs = {
        'portfolio_company':np.array([True]+[False]*27),
        'industry':np.array([False,True]+[False]*26),
        'security___':np.array([False,False,True]+[False]*25),
        'footnotes':np.array([False,False,False]+[True]+[False]*24),
        'investment__interest_rate__maturity':np.array([False,False,False,False]+[True]+[False]*23),
        'aquisition_date':np.array([False]*5+[True]+[False]*22),
        'maturity':np.array([False]*6+[True]+[False]*21), 
        'par_amount___quantity':np.array([False]*8+[True]+[False]*19),  
        'cost':np.array([False]*11+[True]+[False]*16),
        'value':np.array([False]*14+[True]+[False]*13),
        'percentage_of_class':np.array([False]*16+[True]+[False]*11),
    }

# merged_pair_idxs = {}
ex = exceptions()
ex_rows = '|'.join(except_rows())
df,merged_pair_idxs = _clean(file_path,except_rows=ex_rows,merged_pair_idxs=merged_pair_idxs)
display(df)
display(merged_pair_idxs)
index_list = df.apply(
    lambda row:row.astype(str).str.contains(stopping_criterion(None), case=False, na=False).any(),
    axis=1
)
index_list_sum = index_list.sum()
index_list_sum


'portfolio_company'

,1,2,3,4,5,6,7,8,9,10,...,19,20,21,22,23,24,25,26,27,portfolio_company
1,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,03/23/2021,...,1,NaN,NaN,NaN,1,NaN,NaN,*,NaN,ArcLight_Clean_Transition_Corp__200_Clarendon_...
2,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/23/2021,...,-,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,Arctos_NorthStar_Acquisition_Corp.__2021_McKin...
3,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/02/2021,...,18,NaN,NaN,NaN,5,NaN,NaN,*,NaN,Ares_Acquisition_Corp__245_Park_Avenue_44th_Fl...
4,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/04/2021,...,1,NaN,NaN,NaN,-,NaN,NaN,*,NaN,Atlas_Crest_Investment_Corp._II__399_Park_Aven...
5,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/26/2021,...,12,NaN,NaN,NaN,3,NaN,NaN,*,NaN,Austerlitz_Acquisition_Corp._I__1701_Village_C...
6,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/26/2021,...,12,NaN,NaN,NaN,2,NaN,NaN,*,NaN,Austerlitz_Acquisition_Corp._II__1701_Village_...
7,NaN,IT_Services,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/09/2021,...,6,NaN,NaN,NaN,3,NaN,NaN,*,NaN,BigBear.ai_Holdings_Inc.__6811_Benjamin_Frankl...
8,NaN,Healthcare,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,03/02/2021,...,-,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,Biote_Corp.__1875_W._Walnut_Hill_Ln_#100__Irvi...
9,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/24/2021,...,1,NaN,NaN,NaN,1,NaN,NaN,*,NaN,Cartesian_Growth_Corporation__505_5th_Avenue_1...
10,NaN,Special_Purpose_Acquisition_Company,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/12/2021,...,-,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,Catcha_Investment_Corp__Level_42_Suntec_Tower_...


'industry'

,1,3,4,5,6,7,8,9,10,11,...,20,21,22,23,24,25,26,27,portfolio_company,industry
1,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,03/23/2021,NaN,...,NaN,NaN,NaN,1,NaN,NaN,*,NaN,ArcLight_Clean_Transition_Corp__200_Clarendon_...,Special_Purpose_Acquisition_Company
2,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/23/2021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,Arctos_NorthStar_Acquisition_Corp.__2021_McKin...,Special_Purpose_Acquisition_Company
3,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/02/2021,NaN,...,NaN,NaN,NaN,5,NaN,NaN,*,NaN,Ares_Acquisition_Corp__245_Park_Avenue_44th_Fl...,Special_Purpose_Acquisition_Company
4,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/04/2021,NaN,...,NaN,NaN,NaN,-,NaN,NaN,*,NaN,Atlas_Crest_Investment_Corp._II__399_Park_Aven...,Special_Purpose_Acquisition_Company
5,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/26/2021,NaN,...,NaN,NaN,NaN,3,NaN,NaN,*,NaN,Austerlitz_Acquisition_Corp._I__1701_Village_C...,Special_Purpose_Acquisition_Company
6,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/26/2021,NaN,...,NaN,NaN,NaN,2,NaN,NaN,*,NaN,Austerlitz_Acquisition_Corp._II__1701_Village_...,Special_Purpose_Acquisition_Company
7,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/09/2021,NaN,...,NaN,NaN,NaN,3,NaN,NaN,*,NaN,BigBear.ai_Holdings_Inc.__6811_Benjamin_Frankl...,IT_Services
8,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,03/02/2021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,Biote_Corp.__1875_W._Walnut_Hill_Ln_#100__Irvi...,Healthcare
9,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/24/2021,NaN,...,NaN,NaN,NaN,1,NaN,NaN,*,NaN,Cartesian_Growth_Corporation__505_5th_Avenue_1...,Special_Purpose_Acquisition_Company
10,NaN,NaN,Warrants,NaN,8_10,NaN,NaN,NaN,02/12/2021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,Catcha_Investment_Corp__Level_42_Suntec_Tower_...,Special_Purpose_Acquisition_Company


'security___'

...

'percentage_of_class'

,1,3,5,7,9,11,13,14,16,17,...,industry,security___,footnotes,investment__interest_rate__maturity,aquisition_date,maturity,par_amount___quantity,cost,value,percentage_of_class
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,03/23/2021,,400.0,1,1,*
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,02/23/2021,,125.0,-,,*
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,02/02/2021,,20000.0,18,5,*
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,02/04/2021,,1250.0,1,-,*
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,02/26/2021,,12500.0,12,3,*
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,02/26/2021,,12500.0,12,2,*
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IT_Services,Warrants,8_10,,02/09/2021,,8333.0,6,3,*
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Healthcare,Warrants,8_10,,03/02/2021,,400.0,-,,*
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,02/24/2021,,1666.0,1,1,*
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Special_Purpose_Acquisition_Company,Warrants,8_10,,02/12/2021,,166.0,-,,*


,portfolio_company,industry,security___,footnotes,aquisition_date,par_amount___quantity,cost,value,percentage_of_class
0,ArcLight_Clean_Transition_Corp__200_Clarendon_...,Special_Purpose_Acquisition_Company,Warrants,8_10,03/23/2021,400.0,1,1,*
1,Arctos_NorthStar_Acquisition_Corp.__2021_McKin...,Special_Purpose_Acquisition_Company,Warrants,8_10,02/23/2021,125.0,-,NaN,*
2,Ares_Acquisition_Corp__245_Park_Avenue_44th_Fl...,Special_Purpose_Acquisition_Company,Warrants,8_10,02/02/2021,20000.0,18,5,*
3,Atlas_Crest_Investment_Corp._II__399_Park_Aven...,Special_Purpose_Acquisition_Company,Warrants,8_10,02/04/2021,1250.0,1,-,*
4,Austerlitz_Acquisition_Corp._I__1701_Village_C...,Special_Purpose_Acquisition_Company,Warrants,8_10,02/26/2021,12500.0,12,3,*
5,Austerlitz_Acquisition_Corp._II__1701_Village_...,Special_Purpose_Acquisition_Company,Warrants,8_10,02/26/2021,12500.0,12,2,*
6,BigBear.ai_Holdings_Inc.__6811_Benjamin_Frankl...,IT_Services,Warrants,8_10,02/09/2021,8333.0,6,3,*
7,Biote_Corp.__1875_W._Walnut_Hill_Ln_#100__Irvi...,Healthcare,Warrants,8_10,03/02/2021,400.0,-,NaN,*
8,Cartesian_Growth_Corporation__505_5th_Avenue_1...,Special_Purpose_Acquisition_Company,Warrants,8_10,02/24/2021,1666.0,1,1,*
9,Catcha_Investment_Corp__Level_42_Suntec_Tower_...,Special_Purpose_Acquisition_Company,Warrants,8_10,02/12/2021,166.0,-,NaN,*


{'portfolio_company': array([ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]),
 'industry': array([False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]),
 'security___': array([False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]),
 'footnotes': array([False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]),
 'investment__inter

0